In [ ]:
from pyspark.sql import functions as F

CATALOGO = "spotify_analytics"
SCHEMA = "gold"
TABELA = "dm_tempo"
FULL_NAME = f"{CATALOGO}.{SCHEMA}.{TABELA}"

### Configuração

Define catálogo, schema e nome da dimensão temporal (camada Gold - Star Schema).

### DDL - Dimensão Tempo

Cria dimensão temporal para Star Schema:
- **cd_tempo** (PK): Surrogate key no formato YYYYMMDD (ex: 20240515)
- **dt_completa**: Data completa (DATE)
- **nu_ano, nu_mes, nu_dia**: Componentes numéricos
- **nm_mes**: Nome do mês por extenso
- **ds_trimestre, ds_semestre**: Classificações temporais
- **ds_decada**: Década (ex: "2020s")

**Importante**: Dimensão usada no fato ft_track_popularity para análise temporal.

**Decisão Arquitetural**: Usar formato YYYYMMDD como surrogate key ao invés de autoincrement:
1. **Business-friendly**: Fácil interpretação (20240515 = 15/maio/2024)
2. **Particionamento**: Facilita queries com filtro de período
3. **Padrão de mercado**: Kimball recomenda surrogate keys semânticas para dimensões temporais
4. **Performance**: INTEGER (cd_tempo) vs DATE (dt_completa) em JOINs

In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {FULL_NAME} (
    cd_tempo INT NOT NULL COMMENT 'PK: Surrogate key no formato YYYYMMDD (ex: 20240515).',
    dt_completa DATE COMMENT 'Data completa.',
    nu_ano INT COMMENT 'Ano (ex: 2024).',
    nu_mes SMALLINT COMMENT 'Mês (1-12).',
    nu_dia SMALLINT COMMENT 'Dia do mês (1-31).',
    nm_mes STRING COMMENT 'Nome do mês por extenso (ex: Janeiro).',
    ds_trimestre STRING COMMENT 'Trimestre (Q1, Q2, Q3, Q4).',
    ds_semestre STRING COMMENT 'Semestre (S1, S2).',
    ds_decada STRING COMMENT 'Década (ex: 2020s, 2010s).',
    CONSTRAINT pk_dm_tempo PRIMARY KEY (cd_tempo)
)
COMMENT 'Dimensão temporal para análises de tendências ao longo do tempo.';
""")

print(f"✅ Tabela {FULL_NAME} criada com sucesso!")